In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd
import math
from tqdm import tqdm

# HuggingFace Transformer Library
from transformers import *

# Local imports
from attention_graph_util import *




tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased-finetuned-sst-2-english')

data = tfds.load('glue/sst2')

2023-07-10 10:20:38.530702: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/rekha/anaconda3/lib/python3.9/site-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/home/rekha/anaconda3/lib/python3.9/site-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.wa

In [2]:
# Prepare dataset for GLUE as a tf.data.Dataset instance
train_dataset = glue_convert_examples_to_features(data['train'], tokenizer, max_length=128, task='sst-2')
valid_dataset = glue_convert_examples_to_features(data['validation'], tokenizer, max_length=128, task='sst-2')
train_dataset = train_dataset.shuffle(100).batch(32).repeat(2)
valid_dataset = valid_dataset.batch(64)
test_dataset = glue_convert_examples_to_features(data['test'], tokenizer, max_length=128, task='sst-2')
test_dataset = test_dataset.batch(1)

/home/rekha/anaconda3/lib/python3.9/site-packages/transformers/data/processors/glue.py:66: FutureWarning: This function will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING.format("function"), FutureWarning)
/home/rekha/anaconda3/lib/python3.9/site-packages/transformers/data/processors/glue.py:330: FutureWarning: This processor will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING.format("processor"), FutureWarning)
Using label list ['0', '1'] for task sst-2
Using output mode classification for task sst-2
*** 

In [3]:
# Prepare training: Compile tf.keras model with optimizer, loss and learning rate schedule
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

model.evaluate(valid_dataset, steps=1)

1/1 [==============================] - 9s 9s/step - loss: 0.5322 - accuracy: 0.8906


[0.532160758972168, 0.890625]

In [4]:
for layer in model.distilbert.transformer.layer:
    layer.output_attentions = True    
    layer.output_hidden_states = True
    layer.attention.output_attentions = True
    layer.attention.output_hidden_states = True


In [5]:
# Read examples and save attention mats and input gradient scores.
all_examples_grads = []
all_examples_attentions = []
all_examples_x = []
n_examples = 100

In [6]:
model.config.output_attentions = True
model.config.output_hidden_states = True

model.distilbert.transformer.output_attentions = True
model.distilbert.transformer.output_hidden_states = True


In [7]:
for x, y in test_dataset:
    print(x.keys())
    break

dict_keys(['input_ids', 'attention_mask'])


In [9]:
i=0
for x, y in test_dataset:
    i= i+1
    if i==10:
        break

    #print(x['token_type_ids'])
    with tf.GradientTape() as tape:
        inputs_embeds = model.distilbert.embeddings(x['input_ids'])
        tape.watch(inputs_embeds)
        #logits, hidden_states, attentions = model({'attention_mask':x['attention_mask'], 
        #                                            'inputs_embeds':inputs_embeds}, training=False, 
        #                                            output_attentions=True, output_hidden_states=True
        #                                            )
        out = model({'attention_mask':x['attention_mask'], 
                                                   'inputs_embeds':inputs_embeds}, training=False, 
                                                    output_attentions=True, output_hidden_states=True
                                                    )
        logits = out.logits
        hidden_states = out.hidden_states
        attentions = out.attentions
        pindex = tf.argmax(logits, axis=-1)
        true_logits = logits[:,y[0]]
        #print(true_logits)
    grads = tape.gradient(true_logits, inputs_embeds)[0]
    #print("grad", grads.shape)
    
    length = tf.reduce_sum(x['attention_mask'], axis=-1)[0]  
      
    all_examples_grads.append(tf.abs(tf.reduce_sum(grads, -1)[:length]))
    
    _attentions = [att.numpy() for att in attentions]
    attentions_mat = np.asarray(_attentions)[:,0]
        
    cropped_input = x['input_ids'][0, :length]
    all_examples_x.append(cropped_input)
    cropped_attention_mat = attentions_mat[:,:,:length,:length]
    all_examples_attentions.append(cropped_attention_mat)
        

In [10]:
len(all_examples_grads)

399

In [12]:
def offset_convertor(encoded_input_task, task_offset, task_encoder, tokenizer):
    string_part1 = task_encoder.decode(encoded_input_task[:task_offset])
    tokens_part1 = tokenizer.tokenize(string_part1)
    
    return len(tokens_part1)


def get_raw_att_relevance(full_att_mat, input_tokens, layer=-1, output_index=0):
    
    raw_rel = full_att_mat[layer].sum(axis=0)[output_index]/full_att_mat[layer].sum(axis=0)[output_index].sum()
    
    return raw_rel


def get_joint_relevance(full_att_mat, input_tokens, layer=-1, output_index=0):
    att_sum_heads =  full_att_mat.sum(axis=1) / full_att_mat.shape[1]
    #find the attention rollouts
    joint_attentions = compute_joint_attention(att_sum_heads, add_residual=True)
    relevance_attentions = joint_attentions[layer][output_index]
    return relevance_attentions


def get_flow_relevance(full_att_mat, input_tokens, layer, output_index):
    
    input_tokens = input_tokens
    res_att_mat = full_att_mat.sum(axis=1)/full_att_mat.shape[1]
    res_att_mat = res_att_mat + np.eye(res_att_mat.shape[1])[None,...]
    res_att_mat = res_att_mat / res_att_mat.sum(axis=-1)[...,None]

    res_adj_mat, res_labels_to_index = get_adjmat(mat=res_att_mat, input_tokens=input_tokens)
    
    A = res_adj_mat
    res_G=nx.from_numpy_matrix(A, create_using=nx.DiGraph())
    for i in np.arange(A.shape[0]):
        for j in np.arange(A.shape[1]):
            nx.set_edge_attributes(res_G, {(i,j): A[i,j]}, 'capacity')


    output_nodes = ['L'+str(layer+1)+'_'+str(output_index)]
    input_nodes = []
    for key in res_labels_to_index:
        if res_labels_to_index[key] < full_att_mat.shape[-1]:
            input_nodes.append(key)
    
    flow_values = compute_node_flow(res_G, res_labels_to_index, input_nodes=input_nodes, output_nodes=output_nodes, length=full_att_mat.shape[-1])
    
    n_layers = full_att_mat.shape[0]
    length = full_att_mat.shape[-1]
    final_layer_attention = flow_values[(layer+1)*length:, layer*length:(layer+1)*length]
    relevance_attention_flow = final_layer_attention[output_index]

    return relevance_attention_flow

In [13]:
print("compute raw relevance scores ...")
all_examples_raw_relevance = {}
for l in np.arange(0,6):
    all_examples_raw_relevance[l] = []
    for i in tqdm(np.arange(len(all_examples_x))):
        tokens = tokens = tokenizer.convert_ids_to_tokens(all_examples_x[i])
        length = len(tokens)
        attention_relevance = get_raw_att_relevance(all_examples_attentions[i], tokens, layer=l)
        all_examples_raw_relevance[l].append(np.asarray(attention_relevance))

print("compute joint relevance scores ...")
all_examples_joint_relevance = {}
for l in np.arange(0,6):
    all_examples_joint_relevance[l] = []
    for i in tqdm(np.arange(len(all_examples_x))):
        tokens = tokenizer.convert_ids_to_tokens(all_examples_x[i])
        length = len(tokens)
        attention_relevance = get_joint_relevance(all_examples_attentions[i], tokens, layer=l)
        all_examples_joint_relevance[l].append(np.asarray(attention_relevance))
    
print("compute flow relevance scores ...")
all_examples_flow_relevance = {}
for l in np.arange(5,6):
    all_examples_flow_relevance[l] = []
    for i in tqdm(np.arange(len(all_examples_x))):
        tokens = tokenizer.convert_ids_to_tokens(all_examples_x[i])
        length = len(tokens)            
        attention_relevance = get_flow_relevance(all_examples_attentions[i], tokens, layer=l)
        all_examples_flow_relevance[l].append(np.asarray(attention_relevance))

compute raw relevance scores ...


  0%|          | 0/399 [00:00<?, ?it/s]

100%|██████████| 399/399 [00:02<00:00, 175.79it/s]


compute joint relevance scores ...


100%|██████████| 399/399 [00:01<00:00, 261.32it/s]


compute flow relevance scores ...


  0%|          | 0/399 [00:00<?, ?it/s]


TypeError: get_flow_relevance() missing 1 required positional argument: 'output_index'

In [ ]:
for l in np.arange(0,6):
    print("###############Layer ",l, "#############")

    print('raw grad')
    print(all_examples_raw_relevance[l][0].shape, all_examples_grads[0].shape)
    raw_sps_grad = []
    for i in np.arange(len(all_examples_x)):
        sp = spearmanr(all_examples_raw_relevance[l][i],all_examples_grads[i])
        raw_sps_grad.append(sp[0])
 
        
    print(np.mean(raw_sps_grad), np.std(raw_sps_grad))

    
    print('joint grad')
    print(all_examples_joint_relevance[l][0].shape, all_examples_grads[0].shape)
    joint_sps_grad = []
    for i in np.arange(len(all_examples_x)):
        sp = spearmanr(all_examples_joint_relevance[l][i],all_examples_grads[i])
        joint_sps_grad.append(sp[0])

        
    print(np.mean(joint_sps_grad), np.std(joint_sps_grad))

  
    print('flow grad')
for l in np.arange(5,6):
    print(all_examples_joint_relevance[l][0].shape, all_examples_grads[0].shape)
    flow_sps_grad = []
    for i in np.arange(len(all_examples_x)):
        sp = spearmanr(all_examples_flow_relevance[l][i],all_examples_grads[i])
        flow_sps_grad.append(sp[0])
        
    print(np.mean(flow_sps_grad), np.std(flow_sps_grad))

###############Layer  0 #############
raw grad
(10,) (10,)
0.1546719232157473 0.23549466570524266
joint grad
(10,) (10,)
0.1747532362777372 0.14990947384049408
flow grad
###############Layer  1 #############
raw grad
(10,) (10,)
0.17485716719729746 0.1779496014825427
joint grad
(10,) (10,)
0.17533412639102827 0.21118006008452458
flow grad
###############Layer  2 #############
raw grad
(10,) (10,)
0.18401076759142307 0.1555793884889861
joint grad
(10,) (10,)
0.15769953034285264 0.2902687881452012
flow grad
###############Layer  3 #############
raw grad
(10,) (10,)
0.09052739723178496 0.057627671258628
joint grad
(10,) (10,)
0.1763453442753112 0.30548567379412706
flow grad
###############Layer  4 #############
raw grad
(10,) (10,)
0.11866792804125995 0.15310798467445466
joint grad
(10,) (10,)
0.1862251901805654 0.2956835254638234
flow grad
###############Layer  5 #############
raw grad
(10,) (10,)
0.20227427164013048 0.2420496233473882
joint grad
(10,) (10,)
0.23451567945002344 0.2856110